In [2]:
import nltk
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image 
from IPython.core.display import Image, display
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import pymongo
import datetime
import bson
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm
import random
from itertools import permutations
from nltk.tokenize import sent_tokenize, word_tokenize
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [1]:
import aerospike
config = {
    'hosts': [('35.228.136.58', 3000)]
}
client = aerospike.client(config).connect()
data_as_key = ("ids_to_validate", "data", "data")

tags_by_tag_old = client.get(data_as_key)[-1]['tags']

In [4]:
bson_id = "5e8e299bac1658c3c33c1c33"
before = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["grafeed_confirmed"]    
before

['guns',
 's&p',
 'index',
 'us',
 'industrials',
 'sturm ruger & co',
 'american outdoor brands']

In [6]:
def updateAPP(bson_id, new_list):
    before = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["grafeed_confirmed"]    
    before_note = {"date": datetime.datetime.today(),
                   "tags": before,
                  }
    before_set = set(before)
    new_list_set = set(new_list)
    if len(before_set.union(new_list_set))!= 0 and len(before_set.intersection(new_list_set))/len(before_set.union(new_list_set)) != 1:
        db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(bson_id)},
                                    {"$set":{"grafeed_confirmed": new_list,
                                            "fix_grafeed_confirmed": before_note}})

        
        after = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["grafeed_confirmed"]    

In [7]:
updateAPP("5e8e299bac1658c3c33c1c33", ['guns', 's&p', 'index', 'us', 'sturm ruger & co', 'american outdoor brands'])